In [ ]:
# Create initial Dataframe

import os
import pandas as pd
import librosa
import soundfile as sf
from tqdm import tqdm
import numpy as np

#Input directory example
#/Users/michael/Desktop/test_demo

folder_path = input("Input audio path")

#Prepares arrays for dataframe
file_name = []
folder = []
# audio = []
# sample_rate = []
# bit_depth = []
# channels = []
audio_length = []

#Create dataframe
# dataset = pd.DataFrame(columns = ['file_name', 'folder', 'audio', 'sample_rate', 'bit_depth', 'channels', 'audio_length', 'class_id', 'class'])
dataset = pd.DataFrame(columns = ['file_name', 'folder', 'audio_length', 'class_id', 'class'])

# dataset = dataset.astype({'sample_rate': int16})

# gets number of all files in a folder
total = sum([len(files) for r, d, files in os.walk(folder_path)])
pbar = tqdm(total = total)

# Iterates all the wav files in folders and subfolders
# Adds this information to dataframe.
for root, dirs, files in os.walk(folder_path, topdown=False):
    for name in files:
        if ".wav" in name:
            #variables
            base_folder_name = os.path.normpath(os.path.basename(root))
    
            #Load audio file to librosa. 
            # Note that audio_data is already in 32 bit numpy array
            audio_data, sr = librosa.load(root + "/" + name)

            #convert sample rate numpy array
            # sr_data = np.array([sr], dtype=np.int32) 

            #Load audio into soundfile to get 
            # ob = sf.SoundFile(root + "/" + name)

            #upload data into each array
            file_name.append(name) #name of wav file
            folder.append(base_folder_name) #folder location
            # audio.append(audio_data) #raw audio data. 
            # sample_rate.append(sr_data) #sample rate
            # bit_depth.append(ob.subtype) #returns pcm_16 which means 16bit depth
            # channels.append(len(audio_data.shape)) #number of channels
            audio_length.append(audio_data.shape[0]/sr) #length of audio

            pbar.update(1)
        #skips any files thats not a wave
        else:
            pbar.update(1)
            # print(name + " is not a wav file")
            
    
pbar.close()


#Place arrays into dataframe
dataset.file_name = file_name
dataset.folder = folder
# dataset.audio = audio
# dataset.sample_rate = sample_rate
# dataset.bit_depth = bit_depth
# dataset.channels = channels
dataset.audio_length = audio_length

# print(dataset.dtypes) # does not print numpy array type

#Export dataframe
dataset.to_csv (folder_path + '/' + "test.csv", index = False, header=True)


In [ ]:
# Add additonal information to existing dataframe

import os
import sys
import numpy as np
import pandas as pd
import librosa

# /Users/michael/Desktop/test_demo/
# /Users/michael/Desktop/test_demo/test.csv
# /Users/michael/Desktop/test_demo/folder_4

# Input old dataframe
# dataframe_path = input("Input folder path")
dataframe_path = "/Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Dataset/Processed/metadata-v1.8.csv"
#Gets the dir folder of the csv file
dataframe_loc = os.path.dirname(dataframe_path)
old_df = pd.read_csv(dataframe_path)

# Select the folder which you to add to the old dataframe
folder_path = input("Input folder path")


#Prepares the new dataframe to merge with old dataframe
file_name = []
folder = []
audio_length = []

new_df = pd.DataFrame(columns = ['file_name', 'folder', 'audio_length'])

for root, dirs, files in os.walk(folder_path, topdown=False):
    for name in files:
        #Will only process wav files
        if ".wav" in name:
            #Checks for any duplicates
            if name not in old_df.values:
                #variables
                base_folder_name = os.path.normpath(os.path.basename(root))
                audio_data, sr = librosa.load(root + "/" + name)

                #upload data into each array
                file_name.append(name) 
                folder.append(base_folder_name) 
                audio_length.append(audio_data.shape[0]/sr)
                print("Appending " + name + "...")

            else:
                print("Skipped. " + name + " is a duplicate file")
        else:
            print("Skipped. " + name + " is not a wav file")

#Place arrays into dataframe
new_df.file_name = file_name
new_df.folder = folder
new_df.audio_length = audio_length

#merge the old dataframe with new dataframe and export
print("merge old dataframe with new dataframe...")
merge_df = old_df.append(new_df)
merge_df.to_csv (dataframe_loc + '/' + "test.csv", index = False, header=True)
print("exported.")


In [ ]:
# Deleting a column 

import os
import sys
import numpy as np
import pandas as pd

folder_path = "/Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Dataset/Processed"
dataframe_path = "/Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Dataset/Processed/metadata-v1.6.csv"

df = pd.read_csv(dataframe_path)

del df['audio']
del df['sample_rate']
del df['channels']
del df['bit_depth']

df.to_csv(folder_path + '/' + "metadata-v1.6.csv", index = False, header=True)


